In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import sqlite3

In [2]:
Veri_Seti3_201701_In = pd.read_csv('./Dataset 3/Dataset 3_201701_In.csv')

In [3]:
pd.DataFrame(Veri_Seti3_201701_In, index = [0,1,2,3,4])

,CALLER_ID,TIMESTAMP,ID,CITY_ID
0,1100144895,02-01-2017 12:52,568,7
1,1100144895,02-01-2017 20:40,568,7
2,1100144895,02-01-2017 21:41,568,7
3,1100144895,03-01-2017 14:57,828,7
4,1100144895,05-01-2017 10:01,568,7


## Extract information from January dataset

In [4]:
JanuaryIn = np.asarray(Veri_Seti3_201701_In)

In [5]:
length, width = JanuaryIn.shape

In [6]:
print ('there are {} calls recorded in this dataset'.format(length))

there are 998625 calls recorded in this dataset


### Extract timestamp

In [7]:
timestamp = pd.to_datetime(JanuaryIn[0:length,1], format="%d-%m-%Y %H:%M")

### Extract called ID

In [8]:
callerID = JanuaryIn[:,0]

In [9]:
callerID = callerID.astype(str)

### Extract prefix

In [10]:
prefix = []
for i in callerID:
    prefix.append(i[0])

prefix = np.asarray(prefix)
prefix = prefix.astype(int)

In [11]:
unique, counts = np.unique(prefix, return_counts=True)
print ('there are {} refugee calls in this dataset'.format(counts[1]))

there are 758728 refugee calls in this dataset


### Extract city ID

In [12]:
cityID = JanuaryIn[:,3]

## Analyse data

In [92]:
to_analyse = np.unique(callerID[prefix==2])

Accessing date

In [95]:
#Not running
#Not running
evening_location = {}
for i in to_analyse:
    location = np.where(callerID == i)
    location = np.asarray(location)
    location = location.reshape(-1)
    dates = []
    cities = []
    for j in location:
        if timestamp.hour[j] >= 18 or timestamp.hour[j] <= 7:
            dates.append(timestamp.date[j])
            cities.append(cityID[j])
    evening_location[i] = {
    'cities':cities,
    'dates': dates
    }

KeyboardInterrupt: 

Save evening_locations file

In [ ]:
with open("evening_location.pkl", "wb") as fp:   #Pickling
    pickle.dump(evening_location, fp)

In [80]:
with open("evening_location.pkl", "rb") as fp:   # Unpickling
    evening_location = pickle.load(fp)

In [82]:
mobile = [] #find the refugees who spend their time in more that one city in the evening over the month
for i in evening_location:
    unique, counts = np.unique(evening_location[i]['cities'], return_counts=True)
    if len(unique) > 1:
        mobile.append(i)

In [83]:
print ('there are {} callers out ot 50,000 refugees who moved cities'.format(len(mobile)))

there are 1159 callers out ot 50,000 refugees who moved cities


Calculated cities moved between (i.e. were in 'residence' for for more that 4 nights) 

In [86]:
for i in mobile:
    moved = []
    moved.append(evening_location[i]['cities'][0])
    counter = 0
    for j in range(1,len(evening_location[i]['cities'])):
        if evening_location[i]['cities'][j] == evening_location[i]['cities'][j-1]:
            counter += 1
        else:
            counter = 0
        if counter == 4 and evening_location[i]['cities'][j] != moved[-1]:
            moved.append(evening_location[i]['cities'][j])
    evening_location[i]['moved'] = moved

In [88]:
low_mobility = []
medium_mobility = []
high_mobility = []
for i in mobile:
    if len(evening_location[i]['moved']) <=2:
        low_mobility.append(i)
    elif 2 < len(evening_location[i]) <=5:
        medium_mobility.append(i)
    else:
        high_mobility.append(i)

In [91]:
print ('There are {} refugees classed as low mobility'.format(len(low_mobility)))
print ('There are {} refugees classed as medium mobility'.format(len(medium_mobility)))
print ('There are {} refugees classed as high mobility'.format(len(high_mobility)))

There are 997 refugees classed as low mobility
There are 162 refugees classed as medium mobility
There are 0 refugees classed as high mobility


Compare against non-refugee population

In [18]:
not_to_analyse = np.unique(callerID[prefix==1])

In [61]:
non_evening_location = [] #find the cities refugees are in in the evening
for i in non_to_analyse:
    location = np.where(callerID == i)
    location = np.asarray(location)
    location = location.reshape(-1)
    dates = []
    cities = []
    for j in location:
        if hours[j] >= 18 or hours[j] <= 7:
            dates.append(day_month[j])
            cities.append(cityID[j])
    non_evening_location.append(i)
    non_evening_location.append(cities)
    non_evening_location.append(dates)

In [62]:
with open("non_evening_location.txt", "wb") as fp:   #Pickling
    pickle.dump(non_evening_location, fp)

In [19]:
with open("non_evening_location.txt", "rb") as fp:   # Unpickling
    non_evening_location = pickle.load(fp)

IOError: [Errno 2] No such file or directory: 'non_evening_location.txt'

In [64]:
non_to_plot = [] #find the refugees who spend their time in more that one city in the evening over the month
i = 1
while i < len(non_evening_location):
    unique, counts = np.unique(non_evening_location[i], return_counts=True)
    if len(unique) > 1:
        non_to_plot.append(non_evening_location[i-1])
        non_to_plot.append(non_evening_location[i])
        non_to_plot.append(non_evening_location[i+1])
    i += 3

In [76]:
print ('there are {} callers out ot 50,000 refugees who moved cities'.format(len(non_to_plot)/3))

there are 449 callers out ot 50,000 refugees who moved cities


In [77]:
print ('there are {} times as many refugees who moved cities as non-refugees'.format(
    round(float(len(to_plot))/float(len(non_to_plot)),2)))

there are 2.58 times as many refugees who moved cities as non-refugees
